#Load Drive [V]



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Unzip Dataset [V]

In [ ]:
!ls


gdrive	sample_data


In [2]:
!unzip gdrive/MyDrive/minishapenet/val_voxels.zip
!unzip gdrive/MyDrive/minishapenet/train_voxels.zip
!unzip gdrive/MyDrive/minishapenet/val_imgs.zip
!unzip gdrive/MyDrive/minishapenet/train_imgs.zip
!unzip gdrive/MyDrive/minishapenet/test_imgs.zip

Archive:  gdrive/MyDrive/minishapenet/val_voxels.zip
   creating: val_voxels/
   creating: val_voxels/000009/
  inflating: val_voxels/000009/model.mat  
   creating: val_voxels/000022/
  inflating: val_voxels/000022/model.mat  
   creating: val_voxels/000023/
  inflating: val_voxels/000023/model.mat  
   creating: val_voxels/000029/
  inflating: val_voxels/000029/model.mat  
   creating: val_voxels/000031/
  inflating: val_voxels/000031/model.mat  
   creating: val_voxels/000037/
  inflating: val_voxels/000037/model.mat  
   creating: val_voxels/000047/
  inflating: val_voxels/000047/model.mat  
   creating: val_voxels/000061/
  inflating: val_voxels/000061/model.mat  
   creating: val_voxels/000071/
  inflating: val_voxels/000071/model.mat  
   creating: val_voxels/000077/
  inflating: val_voxels/000077/model.mat  
Archive:  gdrive/MyDrive/minishapenet/train_voxels.zip
   creating: train_voxels/
   creating: train_voxels/000003/
  inflating: train_voxels/000003/model.mat  
   creating

#Load libraries + GPU option [V]

##GPU

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


%tensorflow_version 2.x
import tensorflow as tf
import timeit
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

UsageError: Line magic function `%tensorflow_version` not found.


##Other

In [4]:

import os
import re
from scipy import ndimage, misc
from skimage.transform import resize, rescale
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
import cv2
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt
import scipy.io

from scipy.ndimage import zoom
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf

print(tf.__version__)

2.3.1


#Cread File loding funcition [V]

##File loaders

In [5]:
def Collect_image(address):
  image=cv2.imread(address)
  image=resize(image,(256,256))
  image=np.array([image])
  return image
def Collect_voxel(address):
  voxel=scipy.io.loadmat(address)['input']
  voxel=np.array(voxel)
  return voxel

##Address Loader


In [6]:
def pathFinder(path):
  paths=[]
  counti=0
  for(root,dirs,files) in os.walk(path,topdown=True):
    files.sort()
    Tlist_files=files
    if(len(files)==0):
      Tdir_list=dirs
      Tmain_root=root
    if(len(files)!=0):
      counti=counti+1
      for i in range(len(files)):
        path=root+'/'+files[i]
        paths.append(path)
  paths=sorted(paths)
  return paths
# !ls
path=pathFinder('./train_voxels')
print(path)

['./train_voxels\\000003/model.mat', './train_voxels\\000004/model.mat', './train_voxels\\000005/model.mat', './train_voxels\\000007/model.mat', './train_voxels\\000010/model.mat', './train_voxels\\000012/model.mat', './train_voxels\\000014/model.mat', './train_voxels\\000016/model.mat', './train_voxels\\000018/model.mat']


#Create the neural net [V]




In [7]:
#with tf.device('/device:GPU:0'):
input_img=Input(shape=(256,256,3))

l1=tf.keras.layers.Conv2D(32,(3,3),name='l1',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(input_img)
l2=tf.keras.layers.MaxPooling2D(strides=(2,2),name='l2_p')(l1)

l3=tf.keras.layers.Conv2D(256,(3,3),name='l6',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l2)

l4=tf.keras.layers.MaxPooling2D(strides=(2,2),name='l6_p')(l3)
l5=tf.keras.layers.Conv2D(128,(3,3),name='l8',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l4)

l6=tf.keras.layers.MaxPooling2D(strides=(2,2),name='l8_p')(l5)

l7=tf.keras.layers.Conv2D(128,(3,3),name='l10',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l6)

l8=tf.keras.layers.MaxPooling2D(strides=(4,4),name='l10_p')(l7)

l9=tf.keras.layers.MaxPooling2D(strides=(2,2),name='l12_p')(l8)

  #l10=tf.keras.layers.Dense(256 ,name='lfc1',activation='relu')(l10)

  
l11=tf.keras.layers.Reshape((2,2,2,256))(l9)
  

l12=tf.keras.layers.Conv3DTranspose(128,(3,3,3),strides=(1,1,1),name='ld1',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l11)

l13=tf.keras.layers.Conv3DTranspose(128,(1,1,1),strides=(1,1,1),name='ld2',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l12)

l14=tf.keras.layers.Conv3DTranspose(96,(2,2,2),strides=(2,2,2),name='ld3',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l13)

l15=tf.keras.layers.Conv3DTranspose(96,(1,1,1),strides=(1,1,1),name='ld4',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l14)

l16=tf.keras.layers.Conv3DTranspose(96,(2,2,2),strides=(2,2,2),name='ld5',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l15)

l17=tf.keras.layers.Conv3DTranspose(64,(1,1,1),strides=(1,1,1),name='ld6',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l16)

l18=tf.keras.layers.Conv3DTranspose(64,(2,2,2),strides=(2,2,2),name='ld7',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l17)

l19=tf.keras.layers.Conv3DTranspose(64,(1,1,1),strides=(1,1,1),name='ld8',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l18)

l20=tf.keras.layers.Conv3DTranspose(16,(4,4,4),strides=(4,4,4),name='ld9',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l19)

l21=tf.keras.layers.Conv3DTranspose(8,(1,1,1),strides=(1,1,1),name='ld10',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l20)

l22=tf.keras.layers.Conv3DTranspose(1,(2,2,2),strides=(2,2,2),name='ld11',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l21)

l23=tf.keras.layers.Reshape((1,256,256,256))(l22)

#l21=tf.keras.layers.Conv3DTranspose(8,(1,1,1),strides=(1,1,1),name='ld10',activation='relu',activity_regularizer=regularizers.l1(10e-10))(20)

#l22=tf.keras.layers.Conv3DTranspose(1,(2,2,2),strides=(2,2,2),name='ld11',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l21)
  

decoded=l23

  


 
autoencoder=Model(input_img,decoded)
autoencoder.summary()


#with tf.device('/device:GPU:0'):
autoencoder.compile(optimizer='adadelta',loss='binary_crossentropy')

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
l1 (Conv2D)                  (None, 256, 256, 32)      896       
_________________________________________________________________
l2_p (MaxPooling2D)          (None, 128, 128, 32)      0         
_________________________________________________________________
l6 (Conv2D)                  (None, 128, 128, 256)     73984     
_________________________________________________________________
l6_p (MaxPooling2D)          (None, 64, 64, 256)       0         
_________________________________________________________________
l8 (Conv2D)                  (None, 64, 64, 128)       295040    
_________________________________________________________________
l8_p (MaxPooling2D)          (None, 32, 32, 128)      

#create neural net

In [8]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.convolutional import MaxPooling2D
from keras.backend import reshape
EPOCHS = 25
STEPS = 100
LR = 1e-3
BATCH_SIZE = 32
WIDTH = 256
HEIGHT = 256
DEPTH = 3
inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1


# with tf.device('/device:GPU:0'):
model = Sequential()
inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1
model.add(tf.keras.Input(shape=(256, 256, 3)))  
model.add(Conv2D(32,(3,3),name='l1',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
  #model.add(Conv2D(64,(3,3),name='l2',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))##((CONCAT))
model.add(MaxPooling2D(strides=(2,2),name='l2_p'))
  #model.add(Conv2D(96,(3,3),name='l3',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(Conv2D(96,(3,3),name='l4',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(MaxPooling2D(strides=(2,2),name='l4_p'))
  #model.add(Conv2D(256,(3,3),name='l5',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(Conv2D(256,(3,3),name='l6',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(MaxPooling2D(strides=(2,2),name='l6_p'))
  #model.add(Conv2D(128,(3,3),name='l7',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(Conv2D(128,(3,3),name='l8',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(MaxPooling2D(strides=(2,2),name='l8_p'))
  #model.add(Conv2D(256,(3,3),name='l9',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(Conv2D(128,(3,3),name='l10',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(MaxPooling2D(strides=(4,4),name='l10_p'))
  
model.add(MaxPooling2D(strides=(2,2),name='l12_p'))
model.add(Dense(256 ,name='lfc1',activation='relu'))
  

  ###########################SOME LAYER OF ACTIVITY
model.add(tf.keras.layers.Reshape((2,2,2,128)))
model.add(tf.keras.layers.Conv3DTranspose(128,(3,3,3),strides=(1,1,1),name='ld1',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(128,(1,1,1),strides=(1,1,1),name='ld2',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(96,(2,2,2),strides=(2,2,2),name='ld3',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(96,(1,1,1),strides=(1,1,1),name='ld4',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(96,(2,2,2),strides=(2,2,2),name='ld5',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(64,(1,1,1),strides=(1,1,1),name='ld6',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(64,(2,2,2),strides=(2,2,2),name='ld7',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(64,(1,1,1),strides=(1,1,1),name='ld8',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(16,(4,4,4),strides=(4,4,4),name='ld9',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(8,(1,1,1),strides=(1,1,1),name='ld10',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.add(tf.keras.layers.Conv3DTranspose(1,(2,2,2),strides=(2,2,2),name='ld11',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
  #model.add(tf.keras.layers.Reshape((256,256,256)))
  #model.add(tf.keras.layers.Conv3D(64,(3,3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10)))
model.summary()


# Initialize optimizer
opt = Adam(lr=LR, decay=LR / EPOCHS)

# Compile model
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


Using TensorFlow backend.


TypeError: The added layer must be an instance of class Layer. Found: Tensor("input_2:0", shape=(None, 256, 256, 3), dtype=float32)

In [9]:
m_odel=Sequential()
m_odel.add(tf.keras.Input(shape=(256, 256, 3))) 
m_odel.add(Conv2D(32,(3,3),name='l1',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10)))
m_odel.summary()
# Initialize optimizer
opt = Adam(lr=LR, decay=LR / EPOCHS)

# Compile model
m_odel.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

TypeError: The added layer must be an instance of class Layer. Found: Tensor("input_3:0", shape=(None, 256, 256, 3), dtype=float32)

#NOTES



```
# This is formatted as code

  """
  model.add(Conv2D(32, (3,3), padding="same",input_shape=inputShape))
  
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, (3,3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(Conv2D(64, (3,3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, (3,3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(Conv2D(128, (3,3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=chanDim))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation("relu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(n_classes))
 
  model.add(Activation("softmax"))
  """

```





```
# This is formatted as code

with tf.device('/device:GPU:0'):

  EPOCHS = 25
  STEPS = 100
  LR = 1e-3
  BATCH_SIZE = 32
  WIDTH = 256
  HEIGHT = 256
  DEPTH = 3

  inputShape = (HEIGHT, WIDTH, DEPTH)
  chanDim = -1


  input_img=Input(shape=(256,256,3),name='InputThas')
  #odel.add(Conv2D(32, (3,3), padding="same",input_shape=inputShape))
  #Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(input_img)
  l1=tf.keras.layers.Conv2D(96,(7,7),name='l1',padding='same',strides=(1,1),activation='relu',input_shape=inputShape,activity_regularizer=regularizers.l1(10e-10))(input_img)
  l2=tf.keras.layers.Conv2D(96,(3,3),name='l2',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l1)
  l2=tf.keras.layers.MaxPool2D(strides=(2,2),name='l2_p')(l2)
  l3=tf.keras.layers.Conv2D(128,(3,3),name='l3',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l2)
  l4=tf.keras.layers.Conv2D(128,(3,3),name='l4',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l3)############
  l22=tf.keras.layers.Conv2D(128,(1,1),name='l22',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l2)#??
  l4=l4+l22############??
  """
  l4=tf.keras.layers.MaxPool2D(strides=(2,2),name='l4_p')(l4)
  l5=tf.keras.layers.Conv2D(128,(3,3),name='l5',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l4)
  l6=tf.keras.layers.Conv2D(128,(3,3),name='l6',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l5)
  l44=tf.keras.layers.Conv2D(128,(1,1),name='l44',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l4)
  l6=l6+l44
  """
  l6=tf.keras.layers.MaxPool2D(strides=(2,2),name='l6_p')(l6)###
  l7=tf.keras.layers.Conv2D(256,(3,3),name='l7',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l4)###
  l8=tf.keras.layers.Conv2D(256,(3,3),name='l8',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l7)###
  """
  l66=tf.keras.layers.Conv2D(256,(1,1),name='l66',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l4)
  l8=l8+l66
  
  l8=tf.keras.layers.MaxPool2D(strides=(2,2),name='l8_p')(l8)
  """
  l9=tf.keras.layers.Conv2D(256,(3,3),name='l9',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l8)###
  l10=tf.keras.layers.Conv2D(256,(3,3),name='l10',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l9)###
  l88=tf.keras.layers.Conv2D(256,(1,1),name='l88',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l8)#??
  l10=l10+l8#??
  l10=tf.keras.layers.MaxPool2D(strides=(4,4),name='l10_p')(l10)###
  
  """
  l11=tf.keras.layers.Conv2D(256,(3,3),name='l11',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l10)
  l12=tf.keras.layers.Conv2D(256,(3,3),name='l12',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l11)#
  l1010=tf.keras.layers.Conv2D(256,(1,1),name='l1010',padding='same',strides=(1,1),activation='relu',activity_regularizer=regularizers.l1(10e-10))(l10)
  l12=l12+l1010
  l12=tf.keras.layers.MaxPool2D(strides=(2,2),name='l12_p')(l12)
"""
  #l14=tf.reshape(l12,(-1,4*4*256))
  fc=tf.keras.layers.Dense(4096 ,name='lfc1',activation='relu')(l10)###


  #input=tf.reshape(fc,[-1,16,256])
  #tpinput=tf.reshape(input,[-1,16*265-144] )
  #weight=tf.keras.layers.Dense(4096 ,name='lfcw',activation='relu')(tpinput)
  #weight=tf.reshape(weight,[-1,16*16*16])
  #fc=tf.keras.layers.Dense(16*256 ,name='lfc2',activation='relu')(input)####This may fail from here onwards



  ##weights=tf.math.softmax(weight,axis=1,name="SOFTMAX")
  #Input1s=tpinput*weights
  #Input1s=tf.math.reduce_sum(Input1s,axis=1,keepdims=True,name="REDUCED_sUM")
  #Input1s=tf.reshape(Input1s,[-1,4096])
  Vox=tf.keras.layers.Dense(4096 ,name='lfcV',activation='relu')(fc)###
  Vox=tf.reshape(Vox,[-1,4096])
  Vox=tf.reshape(Vox,[-1,2,2,2,256])###
  d1=tf.keras.layers.Conv3DTranspose(256,(3,3,3),strides=(1,1,1),name='ld1',activation='relu',activity_regularizer=regularizers.l1(10e-10))(Vox)
  d2=tf.keras.layers.Conv3DTranspose(256,(1,1,1),strides=(1,1,1),name='ld2',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d1)##
  d00=tf.keras.layers.Conv3DTranspose(256,(1,1,1),strides=(2,2,2),name='ld00',activation='relu',activity_regularizer=regularizers.l1(10e-10))(Vox)
  d2=d2+d00

  d3=tf.keras.layers.Conv3DTranspose(256,(2,2,2),strides=(2,2,2),name='ld3',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d2)##
  d4=tf.keras.layers.Conv3DTranspose(256,(1,1,1),strides=(1,1,1),name='ld4',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d3)#
  d22=tf.keras.layers.Conv3DTranspose(256,(1,1,1),strides=(2,2,2),name='ld22',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d2)
  d4=d4+d22

  d5=tf.keras.layers.Conv3DTranspose(256,(2,2,2),strides=(2,2,2),name='ld5',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d4)
  d6=tf.keras.layers.Conv3DTranspose(256,(1,1,1),strides=(1,1,1),name='ld6',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d5)
  d44=tf.keras.layers.Conv3DTranspose(256,(1,1,1),strides=(2,2,2),name='ld44',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d4)
  d6=d6+d44

  d7=tf.keras.layers.Conv3DTranspose(128,(2,2,2),strides=(2,2,2),name='ld7',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d6)
  d8=tf.keras.layers.Conv3DTranspose(128,(1,1,1),strides=(1,1,1),name='ld8',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d7)##
  d77=tf.keras.layers.Conv3DTranspose(128,(1,1,1),strides=(1,1,1),name='ld77',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d7)
  d8=d8+d77

  """
  d9=tf.keras.layers.Conv3DTranspose(128,(2,2,2),strides=(2,2,2),name='ld9',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d8)#
  d10=tf.keras.layers.Conv3DTranspose(128,(1,1,1),strides=(1,1,1),name='ld10',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d9)##
  d99=tf.keras.layers.Conv3DTranspose(128,(1,1,1),strides=(1,1,1),name='ld99',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d9)
  d10=d10+d99

  """
  d11=tf.keras.layers.Conv3DTranspose(64,(2,2,2),strides=(2,2,2),name='ld11',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d8)
  d12=tf.keras.layers.Conv3DTranspose(64,(1,1,1),strides=(1,1,1),name='ld12',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d11)#
  d1212=tf.keras.layers.Conv3DTranspose(64,(1,1,1),strides=(1,1,1),name='ld1212',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d11)
  d12=d12+d1212


  d13=tf.keras.layers.Conv3DTranspose(32,(2,2,2),strides=(2,2,2),name='ld13',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d12)

  d13=tf.math.sigmoid(d13)
  d13=tf.reshape(d13,[-1,256,256,256,256])

  d14=tf.keras.layers.Conv3D(256,(3,3,3),name='D14',padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(d13)
  decoded=d14



  #tf.keras.layers.Dense(
      #units, activation=None, use_bias=True,
      #kernel_initializer='glorot_uniform',
      #bias_initializer='zeros', kernel_regularizer=None,
      #bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
      #bias_constraint=None, **kwargs
  #)
  #decoded=input

  autoencoder=Model(input_img,decoded)
  autoencoder.summary()
  autoencoder.compile()
  autoencoder.compile(optimizer='adadelta',loss='mean_squared_error')

```



#Load address3 [V]

In [8]:
!ls
Test_Imgs=pathFinder('./test_imgs')
Train_Imgs=pathFinder('./train_imgs')
Train_Voxels=pathFinder('./train_voxels')
Val_Imgs=pathFinder('./val_imgs')
Val_Voxels=pathFinder('./val_voxels')
Size=8
Test_Imgs=Test_Imgs[:Size*12]
Train_Imgs=Train_Imgs[:Size*12]
Train_Voxels=Train_Voxels[:Size]
Val_Imgs=Val_Imgs[:Size*12]

Val_Voxels=Val_Voxels[:Size]
print(len(Test_Imgs))
print(len(Train_Imgs))
print(len(Train_Voxels))
print(len(Val_Imgs))
print(len(Val_Voxels))

gdrive	sample_data  test_imgs	train_imgs  train_voxels  val_imgs  val_voxels
96
96
8
96
8


#Train neural Net [x]

In [9]:
Xpred=[]
Ypred=[]
Xval=[]
Yval=[]

for i in range(Size):
  for j in range(12):
    
    Xpred.append(Collect_image(Train_Imgs[j+i*12]))
    Xval.append(Collect_image(Val_Imgs[j+i*12]))
    Ypred.append(Collect_voxel(Train_Voxels[i]))
    Yval.append(Collect_voxel(Val_Voxels[i]))
print('Done!')


Done!


In [10]:

print(np.array(Xpred).shape)
print(np.array(Ypred).shape)
print(np.array(Xval).shape)
print(np.array(Yval).shape)


(96, 1, 256, 256, 3)
(96, 1, 256, 256, 256)
(96, 1, 256, 256, 3)
(96, 1, 256, 256, 256)


In [ ]:
#with tf.device('/device:GPU:0'):
autoencoder.fit(Xpred, Ypred,
                      validation_data=(Xval, Yval),
                            epochs=50,
                            batch_size=4,steps_per_epoch=len(Xpred) // 6,shuffle=True ,verbose=1)

Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 0.0060

In [ ]:
Data=autoencoder.predict(Xpred[0])
print(Data.shape)

(1, 1, 256, 256, 256)


##Sapce

In [ ]:
#with tf.device('/device:GPU:0'):
#for i in range(len[Xpred]):  #
model.fit(Xpred[0],Ypred[0],validation_data=(Xval[0],Yval[0]),batch_size=12,epochs=12,shuffle=True)


Epoch 1/12
1/1 [==============================] - 1s 720ms/step - loss: 0.0905 - accuracy: 0.0013 - val_loss: 0.2766 - val_accuracy: 0.0029
Epoch 2/12
1/1 [==============================] - 0s 100ms/step - loss: 0.0430 - accuracy: 0.0032 - val_loss: 0.2559 - val_accuracy: 0.0038
Epoch 3/12
1/1 [==============================] - 0s 98ms/step - loss: 0.0405 - accuracy: 0.0037 - val_loss: 0.2449 - val_accuracy: 0.0051
Epoch 4/12
1/1 [==============================] - 0s 97ms/step - loss: 0.0393 - accuracy: 0.0042 - val_loss: 0.2376 - val_accuracy: 0.0061
Epoch 5/12
1/1 [==============================] - 0s 96ms/step - loss: 0.0386 - accuracy: 0.0045 - val_loss: 0.2323 - val_accuracy: 0.0067
Epoch 6/12
1/1 [==============================] - 0s 100ms/step - loss: 0.0382 - accuracy: 0.0044 - val_loss: 0.2283 - val_accuracy: 0.0072
Epoch 7/12
1/1 [==============================] - 0s 98ms/step - loss: 0.0379 - accuracy: 0.0045 - val_loss: 0.2250 - val_accuracy: 0.0079
Epoch 8/12
1/1 [========



```
  history = model.fit(augment.flow(Xpred, Ypred, batch_size=BATCH_SIZE),
                              validation_data=(Xval, Yval),
                              steps_per_epoch=len(Xpred) // BATCH_SIZE,
                              epochs=EPOCHS, 
                              verbose=1)
# This is formatted as code
with tf.device('/device:GPU:1'):
  autoencoder.fit(Xpred, Ypred,
                      validation_data=(Xval, Yval),
                            epochs=4,
                            batch_size=108,shuffle=True )
```



#Predict neural net [x]

In [ ]:
Model=model.predict(Xpred[0])
print(Model.shape)
"""
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

matnp=Model[0][0]



Voxel= np.array(matnp, dtype=bool)
print(Voxel.shape)

colors = np.empty(Voxel.shape, dtype=object)

colors[Voxel] = 'green'

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.voxels(Voxel, facecolors=colors, edgecolor='k')

plt.show() 
"""

(1, 256, 256, 256)


"\nfrom mpl_toolkits.mplot3d import Axes3D\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport scipy.io\n\nmatnp=Model[0][0]\n\n\n\nVoxel= np.array(matnp, dtype=bool)\nprint(Voxel.shape)\n\ncolors = np.empty(Voxel.shape, dtype=object)\n\ncolors[Voxel] = 'green'\n\nfig = plt.figure()\nax = fig.gca(projection='3d')\nax.voxels(Voxel, facecolors=colors, edgecolor='k')\n\nplt.show() \n"

In [ ]:
print(Data[0][0].shape)

(256, 256, 256)


In [ ]:
import numpy as np

def evaluate_voxel_prediction(prediction,gt):
  """  The prediction and gt are 3 dim voxels. Each voxel has values 1 or 0"""
  intersection = np.sum(np.logical_and(prediction,gt))
  union = np.sum(np.logical_or(prediction,gt))
  IoU = intersection / union
  return IoU

mat=scipy.io.loadmat('model.mat')['input'][0]
pred=Data[0][0]
print(pred.shape)
IOU=evaluate_voxel_prediction(pred,mat)
print(IOU)

(256, 256, 256)
1.9626507561112038e-05


#Print a predicted example[x]

# Show accuracy [x]